## add gdp and urbanization data pickle file notebook

Adds GDP (in billion USD) and Urbanization (in population percentage) columns to given tableau and hopkins augmented data.  
Contact ShuliFinley@gmail.com for questions :)

In [438]:
import pandas as pd
import numpy as np
import pickle


Import tableau and hopkins data

In [439]:
# tableau_conf_data = pd.read_csv('../augmented_datasets/tableau_conf_data.csv')
# tableau_death_data = pd.read_csv('../augmented_datasets/tableau_death_data.csv')

# hopkins_conf_data = pd.read_csv('../augmented_datasets/hopkins_conf.csv')
# hopkins_death_data = pd.read_csv('../augmented_datasets/hopkins_death.csv')

# PICKLE_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904.pkl'
# RESULT_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904_GDP_urban.pkl'

PICKLE_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904.pkl'
RESULT_PATH = '../augmented_datasets/pickles/hopkins_conf_gf0904_GDP_urban_tests.pkl'


In [440]:
hopkins_conf = pd.DataFrame()

with open(PICKLE_PATH, 'rb') as file:
#     hopkins_conf = pickle.load(file)
    hopkins_conf = pd.read_pickle(file)
hopkins_conf
    
# hopkins_conf_data.head()

Province_State  \
coordinate           information                                 
(-51.7963, -59.5236) data          Falkland Islands (Malvinas)   
                     avg_d_RH                              NaN   
                     avg_d_precip                          NaN   
                     avg_d_tmp                             NaN   
                     avg_d_wind                            NaN   
...                                                        ...   
(71.7069, -42.6043)  data                            Greenland   
                     avg_d_RH                              NaN   
                     avg_d_precip                          NaN   
                     avg_d_tmp                             NaN   
                     avg_d_wind                            NaN   

                                   Country_Region  avg_m_tmp  avg_m_RH  \
coordinate           information                                         
(-51.7963, -59.5236) data          United Kingdom  10.818182  76.00000   
                     avg_d_RH                 NaN        NaN       NaN   
                     avg_d_precip             NaN        NaN       NaN   
                     avg_d_tmp                NaN        NaN       NaN   
                     avg_d_wind               NaN        NaN       NaN   
...                                           ...        ...       ...   
(71.7069, -42.6043)  data                 Denmark -14.857143  61.12987   
                     avg_d_RH                 NaN        NaN       NaN   
                     avg_d_precip             NaN        NaN       NaN   
                     avg_d_tmp                NaN        NaN       NaN   
                     avg_d_wind               NaN        NaN       NaN   

                                   avg_m_precip  avg_m_wind  Max_Cases  \
coordinate           information                                         
(-51.7963, -59.5236) data                   0.0   29.736364        2.0   
                     avg_d_RH               NaN         NaN        NaN   
                     avg_d_precip           NaN         NaN        NaN   
                     avg_d_tmp              NaN         NaN        NaN   
                     avg_d_wind             NaN         NaN        NaN   
...                                         ...         ...        ...   
(71.7069, -42.6043)  data                   0.0   16.090909       11.0   
                     avg_d_RH               NaN         NaN        NaN   
                     avg_d_precip           NaN         NaN        NaN   
                     avg_d_tmp              NaN         NaN        NaN   
                     avg_d_wind             NaN         NaN        NaN   

                                   first_7 last relevant date  Max_Date  ...  \
coordinate           information                                         ...   
(-51.7963, -59.5236) data              NaN                     4/5/2020  ...   
                     avg_d_RH          NaN                               ...   
                     avg_d_precip      NaN                               ...   
                     avg_d_tmp         NaN                               ...   
                     avg_d_wind        NaN                               ...   
...                                    ...                ...       ...  ...   
(71.7069, -42.6043)  data              NaN                     4/4/2020  ...   
                     avg_d_RH          NaN                               ...   
                     avg_d_precip      NaN                               ...   
                     avg_d_tmp         NaN                               ...   
                     avg_d_wind        NaN                               ...   

                                  3/29/2020  3/30/2020  3/31/2020  4/1/2020  \
coordinate           information                                              
(-51.7963, -59.5236) data               0.0        0.0        0.

Preliminary checks on imported data

In [441]:
# Checking that our starting data has no nan country_region entries in 'data' rows
print('Original df dimensions: {}'.format(hopkins_conf.shape))
check = pd.DataFrame(hopkins_conf.xs('data', level='information', axis=0))
print('Only \'data\' rows dimensions: {}'.format(check.shape))
# check = hopkins_conf[hopkins_conf['information']=='data']
check = check[check['Country_Region'].isna()]

print('\'data\' rows with NaN Country_Region column dimensions: {}'.format(check.shape))

# check = new_hopkins_conf[new_hopkins_conf['GDP'].isna()]
# check = check[check['Country_Region'].notna()]
# print (check.shape[0])

Original df dimensions: (16985, 93)
Only 'data' rows dimensions: (3397, 93)
'data' rows with NaN Country_Region column dimensions: (0, 93)


In [442]:
original_US_states_list = (hopkins_conf[hopkins_conf['Country_Region']=='US']['Province_State'].dropna().unique())
original_US_states_list
# len(original_US_states_list)

array(['American Samoa', 'Guam', 'Northern Mariana Islands',
       'Puerto Rico', 'Virgin Islands', 'Hawaii', 'Florida', 'Texas',
       'Louisiana', 'Mississippi', 'Georgia', 'Alabama', 'Arizona',
       'New Mexico', 'South Carolina', 'California', 'Arkansas',
       'Oklahoma', 'North Carolina', 'Tennessee', 'Missouri', 'Nevada',
       'Kentucky', 'Virginia', 'Kansas', 'Illinois', 'Colorado', 'Utah',
       'West Virginia', 'Indiana', 'Maryland', 'Ohio', 'Delaware',
       'District of Columbia', 'New Jersey', 'Pennsylvania', 'Nebraska',
       'New York', 'Iowa', 'Connecticut', 'Wyoming', 'Massachusetts',
       'Rhode Island', 'Michigan', 'Idaho', 'Oregon', 'Wisconsin',
       'South Dakota', 'New Hampshire', 'Vermont', 'Maine', 'Minnesota',
       'Montana', 'Washington', 'North Dakota', 'Alaska'], dtype=object)

Import gdp per country and per state data

In [443]:
# original_columns = list(hopkins_conf.columns)
# original_columns.append('GDP')
# original_columns.append('Urbanization')

# # print(original_columns)

# prev_GDP = original_columns.index('GDP')
# prev_urban = original_columns.index('Urbanization')
# new_GDP = original_columns.index('Country_Region') + 1
# new_urban = new_GDP + 1
# original_columns.insert(new_GDP, original_columns.pop(prev_GDP))
# original_columns.insert(new_urban, original_columns.pop(prev_urban))

# print (original_columns)

In [444]:
# source: https://data.worldbank.org/indicator/NY.GDP.PCAP.CD  - in current USD
gdp_country_path = '../external_datasets/GDP_per_capita_countries.csv'

# source: https://en.wikipedia.org/wiki/List_of_U.S._states_by_GDP_per_capita#cite_note-3
gdp_us_state_path = '../external_datasets/GDP_per_capita_states.csv'

#source: https://en.wikipedia.org/wiki/List_of_Chinese_administrative_divisions_by_GDP_per_capita
gdp_china_province_path = '../external_datasets/GDP_per_capita_china_provinces.csv'

# source: http://wdi.worldbank.org/table/4.2# (2018)
urban_country_path = '../external_datasets/urbanization_data.csv'

# source: US census bureau (most updated was is from 2010)
urban_us_state_path = '../external_datasets/urbanization_states.csv'

# source: https://en.wikipedia.org/wiki/Urbanization_in_China
urban_china_province_path = '../external_datasets/urbanization_china_provinces.csv'

# source: https://covidtracking.com/
tests_us_states_path = '../external_datasets/tests_us_states.csv'

# ======== MEDIAN AGES ========
# source: https://worldpopulationreview.com/states/median-age-by-state/
median_age_per_US_state_2020_path = '../external_datasets/median_age_per_US_state_2020.csv'
# source: https://ourworldindata.org/grapher/median-age?year=2020&time=2020
median_age_per_countries_2020_path = '../external_datasets/median_age_per_countries_2020.csv'

# ======== Democracy Index by country 2019 ========
# source: https://en.wikipedia.org/wiki/Democracy_Index#cite_note-index2019-7
democracy_index_by_country_path = '../external_datasets/democracy_index_by_country.csv'

In [445]:
def standardize_us_states_names(df):
    df = df.replace({
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
})
    
    return df

In [572]:
# # source: http://wdi.worldbank.org/table/4.2# (2018)
gdp_country = pd.read_csv(gdp_country_path)

# print(gdp_country.tail(20))
gdp_country.columns=['Country_Region', 'GDP']

# # source: https://www.statista.com/statistics/248023/us-gross-domestic-product-gdp-by-state/ 
gdp_us_state = pd.read_csv(gdp_us_state_path)
gdp_us_state.columns = ['Province_State', 'GDP']

gdp_china_province = pd.read_csv(gdp_china_province_path)
gdp_china_province.columns = ['Province_State', 'GDP']

# US states testing data
tests_us_state = pd.read_csv(tests_us_states_path)
tests_us_state.columns=['Province_State', 'Num of Tests']

# ======== MEDIAN AGES ========
# Median age per US state data
median_age_state_col_list = ['State', 'MedianAge']
median_age_per_US_state_2020 = pd.read_csv(median_age_per_US_state_2020_path, usecols = median_age_state_col_list)
median_age_per_US_state_2020.columns = ['Province_State', 'Median Age']

# Median age per country data
median_age_per_countries_2020 = pd.read_csv(median_age_per_countries_2020_path)
median_age_per_countries_2020.columns = ['Country_Region', 'code', 'year', 'Median Age']
median_age_per_countries_2020 = median_age_per_countries_2020.drop(['code'], axis = 1)
median_age_per_countries_2020 = median_age_per_countries_2020[(median_age_per_countries_2020['year'] == 2020) & (median_age_per_countries_2020['Country_Region'] != 'United States')]
median_age_per_countries_2020 = median_age_per_countries_2020.drop(['year'], axis = 1)
median_age_per_countries_2020

# ======== Democracy Index by country 2019 ========
democracy_col_list = ['Country', 'Score']
democracy_index_by_country = pd.read_csv(democracy_index_by_country_path, usecols = democracy_col_list)
democracy_index_by_country.columns = ['Country_Region', 'Democracy']
democracy_index_by_country

,Country_Region,Democracy
0,Norway,9.87
1,Iceland,9.58
2,Sweden,9.39
3,New Zealand,9.26
4,Finland,9.25
...,...,...
163,Syria,1.43
164,Central African Republic,1.32
165,Democratic Republic of the Congo,1.13
166,North Korea,1.08


In [397]:
set(original_US_states_list).symmetric_difference(list(median_age_per_US_state_2020['Province_State']))

# OPTION B:
# for state in original_US_states_list:
#     if not (state in list(median_age_per_US_state_2020['State'])):
#         print (state)

{'American Samoa',
 'Guam',
 'Northern Mariana Islands',
 'Puerto Rico',
 'Virgin Islands'}

In [571]:
# Missing info (excluding Northern Mariana Islands) from median_age_per_countries_2020_path
# Missing info for Northern Mariana Islands from https://en.wikipedia.org/wiki/List_of_countries_by_median_age
missing_median_age_values = {
    'Province_State': ['American Samoa', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands'], 'Median Age': [22, 31.4, 33.6, 38.2, 42.2]
}


# Saint Vincent and the Grenadines (7.9, based on corruption between South Korea and Cabo Verde)
# Seychelles (7.9, based on corruption between Chile and Taiwan)
# Sao Tome and Principe (6.8, based on corruption between Croatia and Argentina)
# Maldives (4, based on corruption between Guinea and Mexico)
# Brunei (7.16, based on corruption of Malaysia)
# Somalia (1, based on corruption index last 3 years)
# Holy See (7.52, same as Italy)
# Grenada (7.3, based on corruption between Italy and Malaysia)
# Belize (7.02, based on list of freedoms indices)
# Dominica (7.78, based on list of freedoms indices)
# Monaco (8.12, same as France)
missing_democracy_values = {
    'Country_Region': ['Saint Vincent and the Grenadines', 'Seychelles', 'Sao Tome and Principe', 'Maldives', 'Brunei', 'Somalia', 'Holy See', 'Grenada', 'Belize', 'Dominica', 'Monaco'],
    'Democracy': [7.9, 7.9, 6.8, 4, 7.16, 1, 7.52, 7.3, 7.02, 7.78, 8.12]
}

In [570]:
# Appending missing_median_age_values 
missing_median_ages_df = pd.DataFrame.from_dict(missing_median_age_values)
median_age_per_US_state_2020 = median_age_per_US_state_2020.append(missing_median_ages_df, ignore_index=True)



,Country_Region,Democracy,Median Age
0,Norway,9.87,NaN
1,Iceland,9.58,NaN
2,Sweden,9.39,NaN
3,New Zealand,9.26,NaN
4,Finland,9.25,NaN
...,...,...,...
196,Holy See,7.52,NaN
197,Grenada,7.3,NaN
198,Belize,7.02,NaN
199,Dominica,7.78,NaN


In [573]:
# Appending missing_democracy_values 
missing_democracy_values_df = pd.DataFrame.from_dict(missing_democracy_values)
democracy_index_by_country = democracy_index_by_country.append(missing_democracy_values_df, ignore_index=True)
democracy_index_by_country

,Country_Region,Democracy
0,Norway,9.87
1,Iceland,9.58
2,Sweden,9.39
3,New Zealand,9.26
4,Finland,9.25
...,...,...
174,Holy See,7.52
175,Grenada,7.3
176,Belize,7.02
177,Dominica,7.78


In [455]:
# original_countries_list = (hopkins_conf['Country_Region'].dropna().unique())
# original_countries_list = standardize_original_names(original_countries_list)

# democracy_index_by_country_stand = standardize_external_names(democracy_index_by_country)
# set(original_countries_list).symmetric_difference(list(democracy_index_by_country_stand['Country_Region']))

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [401]:
gdp_all_provinces = gdp_us_state.append(gdp_china_province, ignore_index=True)
gdp_all_provinces

,Province_State,GDP
0,Alabama,45219.0
1,Alaska,73205.0
2,Arizona,48055.0
3,Arkansas,42454.0
4,California,74205.0
...,...,...
81,Guizhou,6233.0
82,Yunnan,5612.0
83,Gansu,4735.0
84,Hong Kong,48675.0


In [402]:
'Macau' in gdp_country.Country_Region[0]

False

Import urbanization per country and per state data

In [403]:
# source: http://wdi.worldbank.org/table/4.2# (2018)
urban_country = pd.read_csv(urban_country_path)
urban_country.columns=['Country_Region', 'Urbanization']

# source: US census bureau (most updated was is from 2010)
urban_us_state = pd.read_csv(urban_us_state_path)
urban_us_state.columns = ['Province_State', 'Urbanization']


# source: https://en.wikipedia.org/wiki/Urbanization_in_China#cite_note-10
urban_china_province = pd.read_csv(urban_china_province_path)
urban_china_province.columns = ['Province_State', 'Urbanization']

In [404]:
urban_all_provinces = urban_us_state.append(urban_china_province, ignore_index=True)
urban_all_provinces
gdp_country[gdp_country['GDP'].isna()]

,Country_Region,GDP
0,Aruba,NaN
25,Bermuda,NaN
36,Channel Islands,NaN
50,Cayman Islands,NaN
76,Faroe Islands,NaN
82,Gibraltar,NaN
89,Greenland,NaN
106,Isle of Man,NaN
108,Not classified,NaN
145,St. Martin (French part),NaN


In [405]:
def china_check(urban_or_gdp):
    check = hopkins_conf[hopkins_conf['Country_Region']=='China']
    china_original_prov = set(check['Province_State'])
    china_gdp_prov = set(gdp_china_province.Province_State)

    orig_minus_gdp = china_original_prov.difference(china_gdp_prov)
    gdp_minus_orig = china_gdp_prov.difference(china_original_prov)

    china_original_prov = sorted(set(check['Province_State']))
    china_gdp_prov = sorted(set(gdp_china_province.Province_State))

    test_list = [china_gdp_prov, china_original_prov]
    for x, y in zip(*test_list): 
        print(x, y) # print(test_list)
    # print(china_gdp_prov)
    # print(china_original_prov)

    print('orig_minus_gdp: {}'.format(orig_minus_gdp))
    print('gdp_minus_orig: {}'.format(gdp_minus_orig))

data preprocessing functions

In [406]:
def standardize_original_names(df):
    df = df.replace({'Cape Verde': 'Cabo Verde',
                     'Timor-Leste': 'East Timor',
                     'Timor Leste': 'East Timor',
                     'Taiwan*': 'Taiwan',
                     'Myanmar': 'Burma'
                    })
    return df

# tableau_conf_data = standardize_original_names(tableau_conf_data)
# tableau_death_data = standardize_original_names(tableau_death_data)
# hopkins_conf_data = standardize_original_names(hopkins_conf_data)
# hopkins_death_data = standardize_original_names(hopkins_death_data)


In [407]:
index_dict = {
    'gdp_country': 'Country_Region',
    'gdp_us_state': 'Province_State',
    'gdp_china_province': 'Province_State',
    'tests_us_state' : 'Province_State',
    'urban_country': 'Country_Region',
    'urban_us_state': 'Province_State',
    'urban_china_province': 'Province_State'
}


# # setting indices for more convenient access in lambda funtion in add_gdp_urban function
# gdp_country = gdp_country.set_index(index_dict['gdp_country'])
# gdp_us_state = gdp_us_state.set_index(index_dict['gdp_us_state'])
# urban_country = urban_country.set_index(index_dict['urban_country'])
# urban_us_state = urban_us_state.set_index(index_dict['urban_us_state'])

In [512]:
def standardize_external_names(external_df):
    external_df = external_df.replace({
        'United States': 'US',
        'Iran, Islamic Rep.': 'Iran',
        'Congo, Rep.': 'Congo (Brazzaville)',
        'Congo, Dem. Rep.': 'Congo (Kinshasa)',
        'Republic of the Congo': 'Congo (Brazzaville)',
        'Democratic Republic of the Congo': 'Congo (Kinshasa)',
        'St. Lucia': 'Saint Lucia',
        'Czech Republic':'Czechia',
        'Kyrgyz Republic': 'Kyrgyzstan',
        'Egypt, Arab Rep.': 'Egypt',
        'Syrian Arab Republic': 'Syria',
        'Cape Verde': 'Cabo Verde',
        'Brunei Darussalam': 'Brunei',
        'Slovak Republic': 'Slovakia',
        'Russian Federation': 'Russia',
        'Korea, Rep.': 'Korea, South',
        'South Korea[n 2]': 'Korea, South',
        'Timor Leste': 'East Timor',
        'Timor-Leste': 'East Timor',
        'Venezuela, RB': 'Venezuela',
        'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
        'Gambia, The': 'Gambia',
        'St. Kitts and Nevis': 'Saint Kitts and Nevis',
        'Myanmar': 'Burma',
        'Bahamas, The': 'Bahamas',
        'Cape Verde': 'Cabo Verde',
        'Timor-Leste': 'East Timor',
        'Timor Leste': 'East Timor',
        'Taiwan*': 'Taiwan',
        'Myanmar': 'Burma',
        'Ivory Coast': 'Cote d\'Ivoire',
        'Palestine': 'West Bank and Gaza'
       })
    return external_df

# gdp_country = standardize_external_names(gdp_country)
# urban_country = standardize_external_names(urban_country)

this is where the magic happens

In [574]:
# def add_gdp_urban(original, gdp_country, gdp_us_state, gdp_china_province, urban_country, urban_us_state, urban_china_province):
def add_gdp_urban(original, gdp_country, gdp_us_state, urban_country, urban_us_state, tests_us_state, median_age_per_countries_2020, median_age_per_US_state_2020, democracy_index_by_country):
    
    original = standardize_external_names(original)

    gdp_country = standardize_external_names(gdp_country)
    urban_country = standardize_external_names(urban_country)
    tests_us_state = standardize_us_states_names(tests_us_state)
    median_age_per_countries_2020 = standardize_external_names(median_age_per_countries_2020)
    democracy_index_by_country = standardize_external_names(democracy_index_by_country)

    # setting indices for more convenient access in lambda funtion below
    gdp_country = gdp_country.set_index(index_dict['gdp_country'])
    gdp_us_state = gdp_us_state.set_index(index_dict['gdp_us_state'])
#     gdp_china_province = gdp_us_state.set_index(index_dict['gdp_china_province'])
    urban_country = urban_country.set_index(index_dict['urban_country'])
    urban_us_state = urban_us_state.set_index(index_dict['urban_us_state'])
#     urban_china_province = urban_china_province.set_index(index_dict['urban_china_state'])

# 
    tests_us_state = tests_us_state.set_index('Province_State')
    median_age_per_US_state_2020 = median_age_per_US_state_2020.set_index('Province_State')
    median_age_per_countries_2020 = median_age_per_countries_2020.set_index('Country_Region')
    democracy_index_by_country = democracy_index_by_country.set_index('Country_Region')

    original_countries_list = hopkins_conf['Country_Region'].dropna().unique()    
    
#     for df in [gdp_country, urban_country]:
#         df.set_index('Country_Region')
    
#     for df in [gdp_us_state, urban_us_state, tests_us_state]:
#         df.set_index('Province_State')
        

    # if there is no state data, take country data, and if there is also no country data, put NaN
    original['GDP'] = original.apply(lambda row: gdp_us_state.loc[row['Province_State'],'GDP'] \
                                     if row['Province_State'] in list(gdp_us_state.index) \
                                     else (gdp_country.loc[row['Country_Region'],'GDP'] \
                                           if row['Country_Region'] in list(gdp_country.index) \
                                           else np.NaN), axis=1) 
    
    original['Urbanization'] = original.apply(lambda row: urban_us_state.loc[row['Province_State'],'Urbanization'] \
                                              if row['Province_State'] in list(urban_us_state.index) \
                                              else (urban_country.loc[row['Country_Region'],'Urbanization'] \
                                                    if row['Country_Region'] in list(urban_country.index) \
                                                    else np.NaN), axis=1)
    
    original['Num of Tests'] = original.apply(lambda row: tests_us_state.loc[row['Province_State'],'Num of Tests'] \
                                              if row['Province_State'] in list(tests_us_state.index) \
                                              else np.NaN, axis=1)
    
    original['Median Age'] = original.apply(lambda row: median_age_per_US_state_2020.loc[row['Province_State'],'Median Age'] \
                                            if row['Province_State'] in list(median_age_per_US_state_2020.index) \
                                            else (median_age_per_countries_2020.loc[row['Country_Region'], 'Median Age'] \
                                                  if row['Country_Region'] in list(median_age_per_countries_2020.index) \
                                                  else np.NaN), axis=1)
    
    original['Democracy'] = original.apply(lambda row: democracy_index_by_country.loc[row['Country_Region'], 'Democracy'] \
                                           if row['Country_Region'] in list(democracy_index_by_country.index) \
                                           else np.NaN, axis=1)

# new try
#     original['GDP'] = original.apply(lambda row: (gdp_country.loc[row['Country_Region'],'GDP'] if row['Country_Region'] in list(gdp_country.index) else gdp_us_state.loc[row['Province_State'],'GDP'] if row['Province_State'] in list(gdp_us_state.index) else np.NaN), axis=1) 
#     original['Urbanization'] = original.apply(lambda row: (urban_country.loc[row['Country_Region'],'Urbanization'] if row['Country_Region'] in list(urban_country.index) else urban_us_state.loc[row['Province_State'],'Urbanization'] if row['Province_State'] in list(urban_us_state.index) else np.NaN), axis=1)
    
    # if there is no state data, put NaN

    new_columns = list(original.columns)

    prev_GDP = new_columns.index('GDP')
    prev_urban = new_columns.index('Urbanization')
    prev_tests = new_columns.index('Num of Tests')
    prev_median_age = new_columns.index('Median Age')
    prev_democracy = new_columns.index('Democracy')

    new_GDP = new_columns.index('Country_Region') + 1
    new_urban = new_GDP + 1
    new_tests = new_urban + 1
    new_median_age = new_tests + 1
    new_democracy = new_median_age + 1

    new_columns.insert(new_GDP, new_columns.pop(prev_GDP))
    new_columns.insert(new_urban, new_columns.pop(prev_urban))
    new_columns.insert(new_tests, new_columns.pop(prev_tests))
    new_columns.insert(new_median_age, new_columns.pop(prev_median_age))
    new_columns.insert(new_democracy, new_columns.pop(prev_democracy))
    
    return original[new_columns]
#     return original

In [529]:

# pd.set_option('display.max_columns', None)

# new_tableau_conf = add_gdp_urban(tableau_conf_data, gdp_country, gdp_us_state, urban_country, urban_us_state)
# new_tableau_conf.head(5)


In [530]:
# new_tableau_conf.to_csv('../augmented_datasets/tableau_conf_augmented_gdp_urban.csv')


In [531]:
# new_tableau_death = add_gdp_urban(tableau_death_data, gdp_country, gdp_us_state, urban_country, urban_us_state)


In [532]:
# new_tableau_death.to_csv('../augmented_datasets/tableau_death_augmented_gdp_urban.csv')


In [575]:
new_hopkins_conf = add_gdp_urban(hopkins_conf, gdp_country, gdp_all_provinces, urban_country, urban_all_provinces, tests_us_state, median_age_per_countries_2020, median_age_per_US_state_2020, democracy_index_by_country)
new_hopkins_conf

Province_State  \
coordinate           information                                 
(-51.7963, -59.5236) data          Falkland Islands (Malvinas)   
                     avg_d_RH                              NaN   
                     avg_d_precip                          NaN   
                     avg_d_tmp                             NaN   
                     avg_d_wind                            NaN   
...                                                        ...   
(71.7069, -42.6043)  data                            Greenland   
                     avg_d_RH                              NaN   
                     avg_d_precip                          NaN   
                     avg_d_tmp                             NaN   
                     avg_d_wind                            NaN   

                                   Country_Region          GDP  Urbanization  \
coordinate           information                                               
(-51.7963, -59.5236) data          United Kingdom  42943.90227        83.398   
                     avg_d_RH                 NaN          NaN           NaN   
                     avg_d_precip             NaN          NaN           NaN   
                     avg_d_tmp                NaN          NaN           NaN   
                     avg_d_wind               NaN          NaN           NaN   
...                                           ...          ...           ...   
(71.7069, -42.6043)  data                 Denmark  61350.34791        87.874   
                     avg_d_RH                 NaN          NaN           NaN   
                     avg_d_precip             NaN          NaN           NaN   
                     avg_d_tmp                NaN          NaN           NaN   
                     avg_d_wind               NaN          NaN           NaN   

                                   Num of Tests  Median Age Democracy  \
coordinate           information                                        
(-51.7963, -59.5236) data                   NaN   40.799999      8.52   
                     avg_d_RH               NaN         NaN       NaN   
                     avg_d_precip           NaN         NaN       NaN   
                     avg_d_tmp              NaN         NaN       NaN   
                     avg_d_wind             NaN         NaN       NaN   
...                                         ...         ...       ...   
(71.7069, -42.6043)  data                   NaN   42.299999      9.22   
                     avg_d_RH               NaN         NaN       NaN   
                     avg_d_precip           NaN         NaN       NaN   
                     avg_d_tmp              NaN         NaN       NaN   
                     avg_d_wind             NaN         NaN       NaN   

                                   avg_m_tmp  avg_m_RH  avg_m_precip  ...  \
coordinate           information                                      ...   
(-51.7963, -59.5236) data          10.818182  76.00000           0.0  ...   
                     avg_d_RH            NaN       NaN           NaN  ...   
                     avg_d_precip        NaN       NaN           NaN  ...   
                     avg_d_tmp           NaN       NaN           NaN  ...   
                     avg_d_wind          NaN       NaN           NaN  ...   
...                                      ...       ...           ...  ...   
(71.7069, -42.6043)  data         -14.857143  61.12987           0.0  ...   
                     avg_d_RH            NaN       NaN           NaN  ...   
                     avg_d_precip        NaN       NaN           NaN  ...   
                     avg_d_tmp           NaN       NaN           NaN  ...   
                     avg_d_wind          NaN       NaN           NaN  ...   

                                   3/29/2020  3/30/2020  3/31/2020 4/1/2020  \
coordinate           information                                              
(-51.7963, -59.5236) data             

In [435]:
new_hopkins_conf[new_hopkins_conf['Country_Region']=='US']

,,Province_State,Country_Region,GDP,Urbanization,Num of Tests,Median Age,avg_m_tmp,avg_m_RH,avg_m_precip,avg_m_wind,...,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020
coordinate,information,,,,,,,,,,,,,,,,,,,,,
"(-14.271, -170.132)",data,American Samoa,US,62794.58565,18.243,20.0,22.0,27.972368,81.934211,0.000000,10.343421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(13.4443, 144.7937)",data,Guam,US,62794.58565,94.780,946.0,31.4,26.877922,74.025974,1.830779,21.722078,...,56.0,58.0,69.0,77.0,82.0,84.0,93.0,112.0,113.0,121.0
"(15.0979, 145.6739)",data,Northern Mariana Islands,US,62794.58565,91.618,47.0,33.6,27.176623,72.714286,1.008571,25.103896,...,0.0,0.0,2.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
"(18.2208, -66.5901)",data,Puerto Rico,US,62794.58565,93.578,8529.0,38.2,25.748052,75.051948,5.960390,14.240260,...,127.0,174.0,239.0,286.0,316.0,316.0,452.0,475.0,513.0,573.0
"(18.3358, -64.8963)",data,Virgin Islands,US,62794.58565,95.721,400.0,42.2,25.875000,72.789474,1.854079,15.832895,...,23.0,30.0,30.0,30.0,30.0,37.0,40.0,42.0,43.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(64.80726247, -146.5692662)",data,Alaska,US,73205.00000,66.000,7830.0,34.0,-16.824675,69.103896,0.201299,4.989610,...,23.0,28.0,30.0,35.0,40.0,42.0,46.0,53.0,53.0,65.0
"(64.90320724, -164.0353804)",data,Alaska,US,73205.00000,66.000,7830.0,34.0,-15.850704,76.704225,0.010704,17.811268,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(65.50815459, -151.3907387)",data,Alaska,US,73205.00000,66.000,7830.0,34.0,-18.372727,69.233766,0.000000,7.766234,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [578]:
dem_test = pd.DataFrame(new_hopkins_conf.xs('data', level='information', axis=0))
dem_test = dem_test[dem_test['Median Age'].isna()]
# dem_test[dem_test['Country_Region'].isin(['', ''])]
dem_test

# need to fix dem_test = dem_test[dem_test['Median Age'].isna()]

# need to fix dem_test = dem_test[Democracy['Median Age'].isna()]
# 
# missing democracy:
# Barbados (, based on list of freedoms indices)
# Bahamas (, based on list of freedoms indices)
# Saint Lucia (, based on corruption of )
# Antigua and Barbuda (, based on corruption of )
# Saint Kitts and Nevis (, based on corruption of )
# Western Sahara (, based on corruption of )
# Andorra (, based on corruption of )
# Kosovo (, based on corruption of )
# San Marino (, based on corruption of )
# Liechtenstein (, based on corruption of )

,Province_State,Country_Region,GDP,Urbanization,Num of Tests,Median Age,Democracy,avg_m_tmp,avg_m_RH,avg_m_precip,...,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020
coordinate,,,,,,,,,,,,,,,,,,,,,
"(-26.5225, 31.4659)",NaN,Eswatini,4145.974112,23.799,NaN,NaN,3.14,24.618667,71.293333,0.000000,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,10.0,10.0
"(-14.271, -170.132)",American Samoa,US,62794.585650,18.243,20.0,NaN,7.96,27.972368,81.934211,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(-8.874217, 125.727539)",NaN,East Timor,2035.533908,30.578,NaN,NaN,7.19,27.653947,87.144737,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"(-4.0383, 21.7587)",NaN,Congo (Kinshasa),561.777182,44.460,NaN,NaN,1.13,26.659740,73.233766,0.000000,...,65.0,81.0,98.0,109.0,134.0,134.0,154.0,154.0,161.0,180.0
"(-4.03829, 21.7587)",NaN,Congo (Brazzaville),2147.769461,66.916,NaN,NaN,3.11,26.659740,73.233766,0.000000,...,19.0,19.0,19.0,19.0,22.0,22.0,22.0,45.0,45.0,45.0
"(13.4443, 144.7937)",Guam,US,62794.585650,94.780,946.0,NaN,7.96,26.877922,74.025974,1.830779,...,56.0,58.0,69.0,77.0,82.0,84.0,93.0,112.0,113.0,121.0
"(15.0979, 145.6739)",Northern Mariana Islands,US,62794.585650,91.618,47.0,NaN,7.96,27.176623,72.714286,1.008571,...,0.0,0.0,2.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
"(15.415, -61.371)",NaN,Dominica,7691.345097,70.483,NaN,NaN,7.78,24.907792,77.532468,0.000000,...,11.0,11.0,12.0,12.0,12.0,12.0,14.0,14.0,15.0,15.0
"(17.357822, -62.782998)",NaN,Saint Kitts and Nevis,19275.418510,30.776,NaN,NaN,NaN,25.601299,75.558442,0.000000,...,2.0,7.0,8.0,8.0,9.0,9.0,9.0,10.0,10.0,11.0


In [ ]:
# Checking that all entries with Country_Region data have gdp and urbanization data

check = new_hopkins_conf[new_hopkins_conf['Urbanization'].isna()]
check = check[check['Country_Region'].notna()]
check
# print (check.shape[0])

# check = new_hopkins_conf[new_hopkins_conf['GDP'].isna()]
# check = check[check['Country_Region'].notna()]
# print (check.shape[0])

In [ ]:
check = new_hopkins_conf[new_hopkins_conf['GDP'].isna()]
check = check[check['Country_Region'].notna()]
check
# print (check.shape[0])


In [ ]:
# Checking that our starting data has no nan country_region entries in 'data' rows
print('Original df dimensions: {}'.format(hopkins_conf.shape))
check = pd.DataFrame(hopkins_conf.xs('data', level='information', axis=0))
print('Only \'data\' rows dimensions: {}'.format(check.shape))
# check = hopkins_conf[hopkins_conf['information']=='data']
check = check[check['Country_Region'].isna()]

print('\'data\' rows with NaN Country_Region column dimensions: {}'.format(check.shape))

# check = new_hopkins_conf[new_hopkins_conf['GDP'].isna()]
# check = check[check['Country_Region'].notna()]
# print (check.shape[0])

In [144]:
# Checking that our starting data has no nan country_region entries in 'data' rows
print('Original df dimensions: {}'.format(new_hopkins_conf.shape))
check = pd.DataFrame(new_hopkins_conf.xs('data', level='information', axis=0))
check
# print('Only \'data\' rows dimensions: {}'.format(check.shape))
# # check = hopkins_conf[hopkins_conf['information']=='data']
# check = check[check['GDP'].isna()]

# print('\'data\' rows with NaN Country_Region column dimensions: {}'.format(check.shape))

# # check = new_hopkins_conf[new_hopkins_conf['GDP'].isna()]
# # check = check[check['Country_Region'].notna()]
# # print (check.shape[0])

NameError: name 'new_hopkins_conf' is not defined

In [ ]:
# hopkins_conf[hopkins_conf.Province_State.notna()]

In [ ]:
check = new_hopkins_conf[new_hopkins_conf['Country_Region'] == 'China']
check.head(20)


In [ ]:
with open(RESULT_PATH, 'wb') as file:
    pickle.dump(new_hopkins_conf, file)

In [ ]:
test = pd.DataFrame()
with open(RESULT_PATH, 'rb') as file:
    test = pd.read_pickle(file)    
test

In [ ]:
# new_hopkins_conf.to_csv('../augmented_datasets/hopkins_conf_augmented_gdp_urban.csv')


In [ ]:
# new_hopkins_death = add_gdp_urban(hopkins_conf, gdp_country, gdp_us_state, urban_country, urban_us_state)


In [ ]:
# new_hopkins_death.to_csv('../augmented_datasets/hopkins_death_augmented_gdp_urban.csv')
